In [4]:
#Processing
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD, FastICA
from sklearn.model_selection import KFold
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
import gc
#Plotting
import seaborn as sns

#Models
import lightgbm as lgb
import xgboost as xgb


## Load the files and get a brief overview


In [5]:
train_df = pd.read_csv("train.csv", index_col="tripid")

test_df = pd.read_csv("test.csv", index_col="tripid")


In [6]:
train_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [7]:
test_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


### Set up train and test X,Y

In [10]:
X_train = train_df.drop(["label"], axis=1)
y_train = np.log1p(train_df["label"].values)

X_test = test_df.drop(["ID"], axis=1)

TypeError: loop of ufunc does not support argument 0 of type str which has no callable log1p method

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17176 entries, 189123628 to 213817296
Data columns (total 13 columns):
additional_fare              16974 non-null float64
duration                     16974 non-null float64
meter_waiting                16974 non-null float64
meter_waiting_fare           16974 non-null float64
meter_waiting_till_pickup    16974 non-null float64
pickup_time                  17176 non-null object
drop_time                    17176 non-null object
pick_lat                     17176 non-null float64
pick_lon                     17176 non-null float64
drop_lat                     17176 non-null float64
drop_lon                     17176 non-null float64
fare                         17039 non-null float64
label                        17176 non-null object
dtypes: float64(10), object(3)
memory usage: 1.8+ MB


In [ ]:
test_df.info()

# Preprocessing

## Prepare data

### Checking for NaN values and removing constant features in the training data

In [ ]:
print("Total Train Features with NaN Values = " + str(train_df.columns[train_df.isnull().sum() != 0].size))
if (train_df.columns[train_df.isnull().sum() != 0].size):
    print("Features with NaN => {}".format(list(train_df.columns[train_df.isnull().sum() != 0])))
    train_df[train_df.columns[train_df.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

In [ ]:
zero_count = []
for col in X_train.columns[2:]:
    zero_count.append([i[1] for i in list(X_train[col].value_counts().items()) if i[0] == 0][0])
    
print('{0} features of 4491 have zeroes in 99% or more samples.'.format(len([i for i in zero_count if i >= 4459 * 0.99])))
print('{0} features of 4491 have zeroes in 98% or more samples.'.format(len([i for i in zero_count if i >= 4459 * 0.98])))
print('{0} features of 4491 have zeroes in 97% or more samples.'.format(len([i for i in zero_count if i >= 4459 * 0.97])))
print('{0} features of 4491 have zeroes in 96% or more samples.'.format(len([i for i in zero_count if i >= 4459 * 0.96])))
print('{0} features of 4491 have zeroes in 95% or more samples.'.format(len([i for i in zero_count if i >= 4459 * 0.95])))

cols_to_drop = [col for col in X_train.columns[2:] if [i[1] for i in list(X_train[col].value_counts().items()) if i[0] == 0][0] >= 4459 * 0.98]

X_train.drop(cols_to_drop, axis=1, inplace=True)
X_test.drop(cols_to_drop, axis=1, inplace=True)

print('\nTrain shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))

In [ ]:
colsToRemove = []
for col in X_train.columns:
    if X_train[col].std() == 0: 
        colsToRemove.append(col)
        
# remove constant columns in the training set
train_df.drop(colsToRemove, axis=1, inplace=True)

# remove constant columns in the test set
test_df.drop(colsToRemove, axis=1, inplace=True) 

print("Removed `{}` Constant Columns\n".format(len(colsToRemove)))
print(colsToRemove)
print('\nTrain shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))

### Removing duplicated columns

In [ ]:
colsToRemove = []
colsScaned = []
dupList = {}

columns = X_train.columns

for i in range(len(columns)-1):
    v = X_train[columns[i]].values
    dupCols = []
    for j in range(i+1,len(columns)):
        if np.array_equal(v, X_train[columns[j]].values):
            colsToRemove.append(columns[j])
            if columns[j] not in colsScaned:
                dupCols.append(columns[j]) 
                colsScaned.append(columns[j])
                dupList[columns[i]] = dupCols
                
# remove duplicate columns in the training set
X_train.drop(colsToRemove, axis=1, inplace=True) 

# remove duplicate columns in the testing set
X_test.drop(colsToRemove, axis=1, inplace=True)

print("Removed `{}` Duplicate Columns\n".format(len(dupList)))
print(dupList)

print('\nTrain shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))

### Drop Sparse Data

In [ ]:
def drop_sparse(train, test):
    flist = [x for x in train.columns if not x in ['ID','target']]
    for f in flist:
        if len(np.unique(train[f]))<2:
            train.drop(f, axis=1, inplace=True)
            test.drop(f, axis=1, inplace=True)
    return train, test

In [ ]:
X_train, X_test = drop_sparse(X_train, X_test)

print('\nTrain shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))

## Add Features

### Sumzeros and Sumvalues

In [ ]:
def add_SumZeros(train, test, features):
    flist = [x for x in train.columns if not x in ['ID','target']]
    if 'SumZeros' in features:
        train.insert(1, 'SumZeros', (train[flist] == 0).astype(int).sum(axis=1))
        test.insert(1, 'SumZeros', (test[flist] == 0).astype(int).sum(axis=1))
    flist = [x for x in train.columns if not x in ['ID','target']]

    return train, test

In [ ]:
# X_train, X_test = add_SumZeros(X_train, X_test, ['SumZeros'])

In [ ]:
def add_SumValues(train, test, features):
    flist = [x for x in train.columns if not x in ['ID','target']]
    if 'SumValues' in features:
        train.insert(1, 'SumValues', (train[flist] != 0).astype(int).sum(axis=1))
        test.insert(1, 'SumValues', (test[flist] != 0).astype(int).sum(axis=1))
    flist = [x for x in train.columns if not x in ['ID','target']]

    return train, test

In [ ]:
# X_train, X_test = add_SumValues(X_train, X_test, ['SumValues'])

### Other Aggregates

In [ ]:
def add_OtherAgg(train, test, features):
    flist = [x for x in train.columns if not x in ['ID','target','SumZeros','SumValues']]
    if 'OtherAgg' in features:
        train['Mean'] = train.mean(axis=1)
        train['Median'] = train.median(axis=1)
        train['Mode'] = train.mode(axis=1)
        train['Max'] = train.max(axis=1)
        train['Var'] = train.var(axis=1)
        train['Std'] = train.std(axis=1)
        
        test['Mean'] = test.mean(axis=1)
        test['Median'] = test.median(axis=1)
        test['Mode'] = test.mode(axis=1)
        test['Max'] = test.max(axis=1)
        test['Var'] = test.var(axis=1)
        test['Std'] = test.std(axis=1)
    flist = [x for x in train.columns if not x in ['ID','target','SumZeros','SumValues']]

    return train, test

### K-Means

In [ ]:
def kmeans(X_Tr,Xte):
    flist = [x for x in X_Tr.columns if not x in ['ID','target']]
    flist_kmeans = []
    for ncl in range(2,11):
        cls = KMeans(n_clusters=ncl)
        cls.fit_predict(X_train[flist].values)
        X_Tr['kmeans_cluster_'+str(ncl)] = cls.predict(X_Tr[flist].values)
        Xte['kmeans_cluster_'+str(ncl)] = cls.predict(Xte[flist].values)
        flist_kmeans.append('kmeans_cluster_'+str(ncl))
    print(flist_kmeans)
    
    return X_Tr,Xte

### PCA

In [ ]:
def pca(X_Tr,Xte):
    flist = [x for x in X_Tr.columns if not x in ['ID','target']]
    n_components = 20
    flist_pca = []
    pca = PCA(n_components=n_components)
    x_train_projected = pca.fit_transform(StandardScaler(X_Tr[flist], axis=0))
    x_test_projected = pca.transform(StandardScaler(X_test[flist], axis=0))
    for npca in range(0, n_components):
        X_Tr.insert(1, 'PCA_'+str(npca+1), x_train_projected[:, npca])
        Xte.insert(1, 'PCA_'+str(npca+1), x_test_projected[:, npca])
        flist_pca.append('PCA_'+str(npca+1))
    print(flist_pca)

In [ ]:
print('\nTrain shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))

In [ ]:
PERC_TRESHOLD = 0.98   ### Percentage of zeros in each feature ###
N_COMP = 97            ### Number of decomposition components ###

print("\nStart decomposition process...")
print("PCA")
pca = PCA(n_components=N_COMP, random_state=17)
pca_results_train = pca.fit_transform(X_train)
pca_results_test = pca.transform(X_test)
print(pca.explained_variance_ratio_)

print("tSVD")
tsvd = TruncatedSVD(n_components=N_COMP, random_state=17)
tsvd_results_train = tsvd.fit_transform(X_train)
tsvd_results_test = tsvd.transform(X_test)

print("ICA")
ica = FastICA(n_components=N_COMP, random_state=17)
ica_results_train = ica.fit_transform(X_train)
ica_results_test = ica.transform(X_test)

print("GRP")
grp = GaussianRandomProjection(n_components=N_COMP, eps=0.1, random_state=17)
grp_results_train = grp.fit_transform(X_train)
grp_results_test = grp.transform(X_test)

print("SRP")
srp = SparseRandomProjection(n_components=N_COMP, dense_output=True, random_state=17)
srp_results_train = srp.fit_transform(X_train)
srp_results_test = srp.transform(X_test)

print("Append decomposition components to datasets...")
for i in range(1, N_COMP + 1):
    X_train['pca_' + str(i)] = pca_results_train[:, i - 1]
    X_test['pca_' + str(i)] = pca_results_test[:, i - 1]
    
    X_train['ica_' + str(i)] = ica_results_train[:, i - 1]
    X_test['ica_' + str(i)] = ica_results_test[:, i - 1]

    X_train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    X_test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    X_train['grp_' + str(i)] = grp_results_train[:, i - 1]
    X_test['grp_' + str(i)] = grp_results_test[:, i - 1]

    X_train['srp_' + str(i)] = srp_results_train[:, i - 1]
    X_test['srp_' + str(i)] = srp_results_test[:, i - 1]
print('\nTrain shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))


In [ ]:
print(tsvd.explained_variance_ratio_)
sum1 = 0
for i in range(len(tsvd.explained_variance_ratio_)):
    sum1 = sum1 + tsvd.explained_variance_ratio_[i]

print(sum1)

In [ ]:
sum = 0
for i in range(len(pca.explained_variance_ratio_)):
    sum = sum + pca.explained_variance_ratio_[i]

print(sum)

## Build models

### LightGBM

In [ ]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': 'rmse',
    "learning_rate": 0.01,
    "num_leaves": 200,
    "feature_fraction": 0.50,
    "bagging_fraction": 0.50,
    'bagging_freq': 4,
    "max_depth": -1,
    "reg_alpha": 0.3,
    "reg_lambda": 0.1,
    #"min_split_gain":0.2,
    "min_child_weight":10,
    'zero_as_missing':True
}
    
#     {'learning_rate':0.008,
#         'boosting_type':'gbdt',
#         'objective':'regression',
#         'metric':'rmse',
#         'sub_feature':0.5,
#         'num_leaves':180,
#         'feature_fraction': 0.5,
#         'bagging_fraction': 0.5,
#         'min_data':50,
#         'max_depth':-1,
#         'reg_alpha': 0.3, 
#         'reg_lambda': 0.1, 
#         'min_child_weight': 10, 
#         'verbose': 1,
#         'nthread':5,
#         'max_bin':512,
#         'subsample_for_bin':200,
#         'min_split_gain':0.0001,
#         'min_child_samples':5
#        }


#     {
#         "objective" : "regression",
#         "metric" : "rmse",
#         "num_leaves" : 40,
#         "learning_rate" : 0.005,
#         "bagging_fraction" : 0.7,
#         "feature_fraction" : 0.5,
#         "bagging_frequency" : 5,
#         "bagging_seed" : 42,
#         "verbosity" : -1,
#         "random_seed": 42
#     }
#     {
#     'task': 'train',
#     'boosting_type': 'dart',
#     'objective': 'regression',
#     'metric': 'rmse',
#     "learning_rate": 0.01, # Try 0.05
#     "num_leaves": 200, # 24
#     "feature_fraction": 0.50, 
#     "bagging_fraction": 0.90,
#     'bagging_freq': 4,
#     "max_depth": -1,
#     "reg_alpha": 0.3,
#     "reg_lambda": 0.1,
#     #"min_split_gain":0.2,
#     "min_child_weight":10,
#     'zero_as_missing':True
#     'nthread': 32,
#     'lambda_l1':0.1, 
#     'lambda_l2':0.1,
    
    
# }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000, 
                      valid_sets=[lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=50, 
                      evals_result=evals_result)
    
    pred_test_y = np.expm1(model.predict(test_X, num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result

### XGboost

In [ ]:
def run_xgb(train_X, train_y, val_X, val_y, test_X):
    params = {'objective': 'reg:linear', 'eval_metric': 'rmse', 'eta': 0.005, 'max_depth': 10, 'subsample': 0.7, 'colsample_bytree': 0.5, 'alpha':0, 'silent': True, 'random_state':5}
# {'objective': 'reg:linear','eval_metric': 'rmse','eta': 0.005,'max_depth': 15,'subsample': 0.7,'colsample_bytree': 0.5,'alpha':0,'random_state':42,'silent': True}
    
    tr_data = xgb.DMatrix(train_X, train_y)
    va_data = xgb.DMatrix(val_X, val_y)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model_xgb = xgb.train(params, tr_data, 2000, watchlist, maximize=False, early_stopping_rounds = 30, verbose_eval=100)
    
    dtest = xgb.DMatrix(test_X)
    xgb_pred_y = np.expm1(model_xgb.predict(dtest, ntree_limit=model_xgb.best_ntree_limit))
    
    return xgb_pred_y, model_xgb

### Training 

In [ ]:
#Split data
dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)


# Training LGB
pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
print("LightGBM Training Completed...")

print("Features Importance...")
gain = model.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model.feature_name(), 
                   'split':model.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:15])

# Training XGB
pred_test_xgb, model_xgb = run_xgb(dev_X, dev_y, val_X, val_y, X_test)
print("XGB Training Completed...")

### Create file with submissions

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')

sub_lgb = pd.DataFrame()
sub_lgb["target"] = pred_test
sub_lgb["ID"] = sub["ID"]
sub_lgb.to_csv("sub_lgb.csv", index=False)

sub_xgb = pd.DataFrame()
sub_xgb["target"] = pred_test_xgb
sub_xgb["ID"] = sub["ID"]
sub_lgb.to_csv("sub_xgb.csv", index=False)


sub["target"] = (sub_lgb["target"] + sub_xgb["target"] )/2

print(sub.head())
sub.to_csv('sub_lgb_xgb.csv', index=False)